In [ ]:
import cv2
import json
import os
import  numpy as np
import time
import glob
#########注意input json路径  
result = "/home/admin/jupyter/Demo/Models/detectron2_bai/outs/out_cascade_mask_rcnn_X_152/inference/my_dataset_test.json"
#"/home/admin/jupyter/Demo/Models/detectron2_bai/outs/out_cascade_mask_rcnn_X_152/inference/my_dataset_test.json"  #input_json
result1 = result # "/home/admin/jupyter/Demo/Models/detectron2_bai/outs/out_cascade_mask_rcnn_X_152_cl4/inference/my_dataset_test.json"  #input_json
result2 = result #"/home/admin/jupyter/Demo/Models/detectron2_bai/outs/out_cascade_mask_rcnn_R_50_CA/inference/my_dataset_test.json"  #input_json
########测试集json###
refjson=  '/home/admin/jupyter/Demo/DataSets/Data/test.json' #"/home/admin/jupyter/Demo/DataSets/Data/test_ts.json"
refdir=  r'/home/admin/jupyter/Demo/DataSets/Data/images_test_json/'#/home/admin/jupyter/Demo/DataSets/Data/images_test_ts_json/' #' 
####################

In [ ]:
################## load results & build category dict###########
start=time.time()
with  open(result, "r") as f:
    results = json.load(f)
print('###Results e.g. %s'% results[:4])
print('!!!Total: %d'%len(results),' cost time s: ',time.time()-start)
#####加
with  open(result1, "r") as f:
    results1 = json.load(f)
with  open(result2, "r") as f:
    results2 = json.load(f)

start=time.time()
with  open(refjson, "r") as f:
    j=json.load(f)
    print('###Test annotations: ',j['annotations'][:4])
    categories_list=j['categories']
    im_list = j['images']
#print('###Test images e.g. %s'%im_list[:4],'\tTotal: %d'%len(im_list),'cost time s:',time.time()-start)
categories_dict={}
for c in categories_list:
    categories_dict[c['id']] = c['name'] #建立字典 id对应 类别
print('Categories: %s'%categories_dict)

In [ ]:
#建立id-name字典
images_dict = {}
start=time.time()
n=0
for im_str in im_list:
    name=im_str['file_name'].split('.')[0]
    with  open(refdir + name + '.json', "r") as f:
            js = json.load(f)
#             print(js)
    if not js.__contains__('xy'): 
        images_dict[im_str['id']] = [js['at_20_Original_image'][0],js['at_20_Original_image'][1],name.split('_')[0]]
    else:
        images_dict[im_str['id']] = [js['xy'][0],js['xy'][1],js['Original_name']] #建立字典 id 对应 原图x,y 原图名字

print('建立字典,共%d个,e.g. %s'%(len(images_dict),images_dict[1]),'cost time s:',time.time()-start) #70s
print(im_list[:5])

In [ ]:
import copy
def chuli5 (bboxes):  ###chuli 2与 4结合
    
    tboxes =  bboxes  
    for i ,box in enumerate(bboxes):
        if bboxes[i]["p"] == 0.0:
            continue        
        for j,tbox in enumerate(tboxes):
            if j==i:
                continue
            if bboxes[j]["p"] == 0.0: # 
                continue
            if box["class"]!=tbox["class"]:   ##加 对不同类 分离处理
                continue
            ##l两筐没关系
            if box["x"]+box['w']<=tbox["x"] or box["y"]+box['h']<=tbox["y"] or box["x"]>=tbox["x"]+tbox["w"] or box["y"]>=tbox["y"]+tbox["h"]: 
                continue
            ##在里面   念珠菌先处理 包含关系 和后面有点不一样
#             if box["class"]== "Candida" and box["x"]>=tbox["x"] and box["y"]>=tbox["y"] and box["x"] + box["w"]<=tbox["x"]+tbox["w"] and box["y"] + box["h"]<=tbox["y"]+tbox["h"] :
#                 if box["p"]>=tbox["p"]+0.2 :  ###  +0.2
#                     bboxes[j]["p"] = 0
#                 else :
#                     bboxes[i]["p"] = 0
#                 continue
            ####得到 iou   公式改进
            in_h = min(box['y']+box['h'],tbox['y']+tbox['h'])-max(box['y'],tbox['y'])
            in_w = min(box['x']+box['w'],tbox['x']+tbox['w'])-max(box['x'],tbox['x'])
            union = box['w']*box['h']+tbox['w']*tbox['h']-in_h*in_w
            iou = in_h*in_w/union
            # 单独对 念珠菌处理 更容易放大操作
            if box["class"]== "Candida":   # 单独对 念珠菌处理 更容易放大操作
                ##在里面
                if box["x"]>=tbox["x"] and box["y"]>=tbox["y"] and box["x"] + box["w"]<=tbox["x"]+tbox["w"] and box["y"] + box["h"]<=tbox["y"]+tbox["h"] :
                    if box["p"]>=tbox["p"]+0.2 :  ###  +0.2
                        bboxes[j]["p"] = 0
                    else :
                        bboxes[i]["p"] = 0
                    continue
                if iou >= 0.95 :  ###0.8         #iou 高                                    
                    if box["p"]>=tbox["p"]:  
                        bboxes[j]["p"] = 0
                    else :
                        bboxes[i]["p"] = 0
                    continue
                elif iou >0.4 and iou < 0.95 and box["class"]==tbox["class"] and box["p"]>0.3 and tbox["p"]>0.3: #iou中等，且同类，且置信度高
                    bboxes[i]["x"] = min(box["x"],tbox["x"])
                    bboxes[i]["y"] = min(box["y"],tbox["y"])
                    bboxes[i]["w"] = abs(box["x"]-tbox["x"])+tbox["w"]
                    bboxes[i]["h"] = abs(box["y"]-tbox["y"])+tbox["h"]
                    bboxes[i]["p"] = max(box["p"],tbox["p"])
                    bboxes[j]["p"] = 0
                    continue
                elif iou >0.4 and iou <0.95 :   #iou中等 且同类，置信度不高
                    if box["p"]>=tbox["p"]:  
                        bboxes[j]["p"] = 0
                    else :
                        bboxes[i]["p"] = 0
                    continue
                else:
                    pass

                ## 对大小框处理 大框包括小框太多
                s1 = box["w"] *box["h"]
                s2 = tbox["w"] *tbox["h"]
                ss = in_h*in_w

                if (ss/s1 >0.6 )and ( s2/s1>3) : ##s1 太小  0.5 改0.8   
                    if tbox["p"]> 0.5:
                        bboxes[i]["p"] = 0
                    elif box["p"]>=tbox["p"]+0.2 :  ###  +0.2
                        bboxes[j]["p"] = 0
                    else :
                        bboxes[i]["p"] = 0
                    continue
                elif (ss/s2 >0.6 )and ( s1/s2>3) :  ##s2 太小
                    if box["p"]> 0.5:
                        bboxes[j]["p"] = 0
                    elif box["p"]+0.2<=tbox["p"] :  ###  +0.2
                        bboxes[i]["p"] = 0
                    else :
                        bboxes[j]["p"] = 0
                    continue
                else:
                    pass
            ############  对其他类的处理
            if iou >= 0.8 :  ###0.8         #iou 高                                    
                if box["p"]>=tbox["p"]:  
                    bboxes[j]["p"] = 0
                else :
                    bboxes[i]["p"] = 0
                continue
            elif iou >0.4 and iou <0.8 and box["p"]>0.4 and tbox["p"]>0.4: #iou中等，且同类，且置信度高
                bboxes[i]["x"] = min(box["x"],tbox["x"])
                bboxes[i]["y"] = min(box["y"],tbox["y"])
                bboxes[i]["w"] = abs(box["x"]-tbox["x"])+tbox["w"]
                bboxes[i]["h"] = abs(box["y"]-tbox["y"])+tbox["h"]
                bboxes[i]["p"] = max(box["p"],tbox["p"])
                bboxes[j]["p"] = 0
                continue
            elif iou >0.4 and iou <0.8 :   #iou中等 且同类，置信度不高
                if box["p"]>=tbox["p"]:  
                    bboxes[j]["p"] = 0
                else :
                    bboxes[i]["p"] = 0
                continue
            else:
                pass
            
    for i ,box in enumerate(bboxes):
        if bboxes[i]["p"] == 0.0:
            continue
        if box["class"]== "Candida":  ##上面全面处理了
            continue
        for j,tbox in enumerate(tboxes):
            if j==i:
                continue
            if bboxes[j]["p"] == 0.0: # 
                continue
            if box["class"]!=tbox["class"]:   ##加 对不同类 分离处理
                continue
            ##l两筐没关系
            if box["x"]+box['w']<=tbox["x"] or box["y"]+box['h']<=tbox["y"] or box["x"]>=tbox["x"]+tbox["w"] or box["y"]>=tbox["y"]+tbox["h"]: 
                continue
            ##在里面 
            if  box["x"]>=tbox["x"] and box["y"]>=tbox["y"] and box["x"] + box["w"]<=tbox["x"]+tbox["w"] and box["y"] + box["h"]<=tbox["y"]+tbox["h"] :
                if tbox["p"]> 0.5:
                    bboxes[i]["p"] = 0
                elif box["p"]>=tbox["p"]+0.2 :  ###  +0.2
                    bboxes[j]["p"] = 0
                else :
                    bboxes[i]["p"] = 0
                continue
            ## 对大小框处理 大框包括小框太多
            s1 = box["w"] *box["h"]
            s2 = tbox["w"] *tbox["h"]
            in_h = min(box['y']+box['h'],tbox['y']+tbox['h'])-max(box['y'],tbox['y'])
            in_w = min(box['x']+box['w'],tbox['x']+tbox['w'])-max(box['x'],tbox['x'])
            ss = in_h * in_w
            
            if (ss/s1 >0.8 )and ( s2/s1>4) : ##s1 太小  0.5 改0.8   
                if tbox["p"]> 0.5:
                    bboxes[i]["p"] = 0
                elif box["p"]>=tbox["p"]+0.2 :  ###  +0.2
                    bboxes[j]["p"] = 0
                else :
                    bboxes[i]["p"] = 0
                continue
            elif (ss/s2 >0.8 )and ( s1/s2>4) :  ##s2 j 小  i 大
                if box["p"]> 0.5:
                    bboxes[j]["p"] = 0
                elif box["p"]+0.2<=tbox["p"] :  ###  +0.2
                    bboxes[i]["p"] = 0
                else :
                    bboxes[j]["p"] = 0
                continue
            else:
                pass
    return bboxes

In [ ]:
def arrays2dicts(boxes,thres=0):
    """
    boxes:a dict of arrays, key as class and each box array:x,y,w,h,p
    return:a list of dicts,and each box dict:x,y,w,h,p,class 
    """
    out = []
    categories={1: 'ASC-H', 2: 'ASC-US', 3: 'HSIL', 4: 'LSIL', 5: 'Candida', 6: 'Trichomonas'}
    counts={'HSIL':0, 'LSIL': 0, 'ASC-H': 0, 'ASC-US': 0,'Candida':0, 'Trichomonas':0}
    if not isinstance(boxes,dict):
        boxes={'all':boxes}
    for c,boxes_ in boxes.items():
        if isinstance(thres,dict):
            threshold=thres[c]
        elif isinstance(thres,float):
#             assert(isinstance(thres,float),'should be dict or float!')
            threshold=thres
        else:
            raise Exception('thres should be dict or float!')
        for i in range(len(boxes_)):
            b = boxes_[i]
            # b x y w h p
            if b[4]<=threshold:
                continue
            box = {}
            cls=categories[b[5]]
            box['x'] = b[0]
            box['y'] = b[1]
            box['w'] = b[2] - box['x']
            box['h'] = b[3] - box['y']
            box['p'] = b[4]
            box['class'] =cls
            out.append(box)
            counts[cls]+=1
            
    return out,counts


def dicts2arrays(boxes):
    """
    boxes:a list of dicts,and each box dict:x,y,w,h,p,class
    return: a dict of arrays, let class as key and each box array:x,y,w,h,p,class
    """
    out = {}
    categories={'ASC-H':1, 'ASC-US':2, 'HSIL':3, 'LSIL':4, 'Candida':5, 'Trichomonas':6}
    for box in boxes:
        # box x y w h p class
        b = np.zeros(6)
        b[0] = box['x']
        b[1] = box['y']
        b[2] = box['x'] + box['w']
        b[3] = box['y'] + box['h']
        b[4] = box['p']
        b[5] = categories[box['class']]
        b = np.expand_dims(b, axis=0)
        if box['class'] not in out:
            out[box['class']]=b
        else:
            out[box['class']] = np.concatenate((out[box['class']], b), axis=0)
    return out


def nms(boxes, threshold=0.5, method='original', sigma=0.5):
    #boxes :an array of x,y,w,h,p
    #     print(boxes.shape)
    # coordinates of bounding boxes
    if (len(boxes))==0:
        return boxes
    s_x = boxes[:, 0]
    s_y = boxes[:, 1]
    e_x = boxes[:, 2]
    e_y = boxes[:, 3]
    # Confidence scores of bounding boxes
    score = boxes[:, 4]
    # Picked bounding boxes
    #     picked_boxes = []
    # Compute areas of bounding boxes
    areas = (e_x - s_x + 1) * (e_y - s_y + 1)
    # Sort by confidence score of bounding boxes
    order = np.argsort(score)

    # Iterate bounding boxes
    while order.size > 0:
        # The index of largest confidence score
        index = order[-1]

        # Pick the bounding box with largest confidence score
        #         picked_boxes.append(bboxes[index])

        # Compute ordinates of intersection-over-union(IOU)
        x1 = np.maximum(s_x[index], s_x[order[:-1]])
        x2 = np.minimum(e_x[index], e_x[order[:-1]])
        y1 = np.maximum(s_y[index], s_y[order[:-1]])
        y2 = np.minimum(e_y[index], e_y[order[:-1]])

        # Compute areas of intersection-over-union
        w = np.maximum(0.0, x2 - x1 + 1)
        h = np.maximum(0.0, y2 - y1 + 1)
        intersection = w * h

        # Compute the ratio between intersection and union
        ratio = intersection / (areas[index] + areas[order[:-1]] - intersection)
        if method == 'gaussian':
            weight = np.exp(-(ratio * ratio) / sigma)
        elif method == 'linear':
            weight = 1 - ratio
        elif method == 'original':
            weight = np.ones_like(ratio)
            weight[ratio > threshold] = 0
        else:
            weight = np.ones_like(ratio)
        weight[ratio > 0.7] = 0
#         print(len(ratio),len(weight),len(order))
        boxes[order[:-1], 4] *= weight
        left = np.where(ratio < threshold)
        order = order[left]
    boxes=boxes[np.argsort(boxes[:, 4]),:]
    return boxes
def nms_by_class(bboxes, threshold=0.5, method='original', sigma=0.5,score_thres=0):
    """
    bboxes: a list of boxes,and each box :x,y,w,h,p,class
    """
    counts={'HSIL':0, 'LSIL': 0, 'ASC-H': 0, 'ASC-US': 0,'Candida':0, 'Trichomonas':0}
    if len(bboxes) == 0:
        return [],counts
    # Bounding boxes
    cboxes = dicts2arrays(bboxes)
    oboxes={}
    total_nonzero=0
    log=''
    for c,boxes in cboxes.items():
        oboxes[c]=nms(boxes, threshold=threshold, method=method, sigma=sigma)
        log+='%s:(non_zero:%d,max:%3f) ; '%(c,np.count_nonzero(oboxes[c][:,-2]),oboxes[c][-1][-2])
    out,counts=arrays2dicts(oboxes,score_thres)
    print('%s\nFiltered %s'%(log,counts))
    return out,counts
def nms_all(bboxes, threshold=0.5, method='original', sigma=0.5,score_thres=0):
    """
    bboxes: a list of boxes,and each box :x,y,w,h,p,class
    """
    counts={'HSIL':0, 'LSIL': 0, 'ASC-H': 0, 'ASC-US': 0,'Candida':0, 'Trichomonas':0}
    if len(bboxes) == 0:
        return [],counts
    # Bounding boxes
    cboxes = dicts2arrays(bboxes)
    total_nonzero=0
    for i,(_,boxes) in enumerate(cboxes.items()):
        if i==0:
            oboxes=boxes
        else:
            oboxes=np.concatenate((oboxes, boxes), axis=0)
    
    oboxes=nms(oboxes, threshold=threshold, method=method, sigma=sigma)
    out,counts=arrays2dicts(oboxes,score_thres)
    print('Filtered %s'%(counts))
    return out,counts

In [ ]:
# nms & merge 
outpath='./out_jsons/'#'./out_val_20_jsons/' './out_jsons/'
if not os.path.exists(outpath):
    os.makedirs(outpath)
start=time.time()
count=0
total=0
total_by_cls={'HSIL':0, 'LSIL': 0, 'ASC-H': 0, 'ASC-US': 0,'Candida':0, 'Trichomonas':0}
merge=False#####
sub_path1='./d191223/out_jsons/'
# score_thres={'all':0.2,'HSIL':0.5, 'LSIL': 0.5, 'ASC-H': 0.3, 'ASC-US': 0.3,'Candida':0.2, 'Trichomonas':0.2} # 1213_cls_4
# score_thres={'all':0.2,'HSIL':0.6, 'LSIL': 0.2, 'ASC-H': 0.6, 'ASC-US': 0.2,'Candida':0.2, 'Trichomonas':0.2} #1215_cls_2
# score_thres={'all':0.2,'HSIL':0.6, 'LSIL': 0.2, 'ASC-H': 0.6, 'ASC-US': 0.6,'Candida':0.2, 'Trichomonas':0.2}#1217_cls_3
# score_thres={'all':0.0,'HSIL':0.3, 'LSIL': 0.3, 'ASC-H': 0.3, 'ASC-US': 0.3,'Candida':0.3, 'Trichomonas':0.3}#1217_cls_6 建议 'Candida':0.5 考虑 'Trichomonas':0.2~0.3
# score_thres={'all':0.0,'HSIL':0.3, 'LSIL': 0.3, 'ASC-H': 0.2, 'ASC-US': 0.2,'Candida':0.2, 'Trichomonas':0.2} #1224_cls_4
score_thres={'all':0.0,'HSIL':0.5, 'LSIL': 0.3, 'ASC-H': 0.4, 'ASC-US': 0.4,'Candida':0.5, 'Trichomonas':0.2}
for name,bboxes in out.items():
    count+=1
    print('No.%d %s.json total:%d'%(count,name,len(bboxes)))
    print('nms_by_cls_1')
    boxes,count_by_cls=nms_by_class(bboxes,0.4,method='linear',score_thres=score_thres) #  'gaussian'   ''   'original'   'linear'
    num=len(boxes)
    total_by_cls=dict(Counter(total_by_cls)+Counter(count_by_cls))
    print('After nms_1 %s.json->total:%d'%(name,num))
    #####mergerd with subjsons#################
    if merge:
        sub_jsons=os.path.join(sub_path1,name+'.json')
        if os.path.exists(sub_jsons):
            with open(sub_jsons,'r') as f:
                subs=json.load(f)
                #只取滴虫和念珠菌
                subs,count_by_cls=nms_by_class(subs,0.4,method='',score_thres={'all':0.0,'HSIL':1, 'LSIL': 1, 'ASC-H': 1, 'ASC-US': 1,'Candida':0, 'Trichomonas':0} )
                boxes+=subs
                print('merged with %d ,get total:%d'%(len(subs),len(boxes)))
                print('nms_by_cls_2')
#                 boxes,count_by_cls=nms_by_class(boxes,0.4,method='linear',score_thres=score_thres) #  'gaussian'   ''   'original'   'linear'
                boxes,count_by_cls=nms_by_class(boxes,0.4,method='',score_thres=score_thres)
                num=len(boxes)
                total_by_cls=dict(Counter(total_by_cls)+Counter(count_by_cls))
                print('After nms_2 %s.json->total:%d'%(name,num))
#     print('nms_all')
#     boxes,count_by_cls=nms_all(boxes,0.4,method='linear',score_thres=score_thres) #  'gaussian'   ''   'original'   'linear'   
#     num=len(boxes)
#     total_by_cls=dict(Counter(total_by_cls)+Counter(count_by_cls))
#     print('After nms_all %s.json->total:%d'%(name,num))
    ############################################
    total+=num
    outjson=os.path.join(outpath,name+'.json')
    with open(outjson,'w') as f:
        json.dump(boxes,f)
print('get nms outs, total: %d %s, cost time:%f'%(total,total_by_cls,time.time()-start))

In [ ]:
#  返回原图坐标
def get_out(results2):
    start=time.time()
    lost2=[]
    out2={}
    i=1
    count=0
    for res in results2:


        d={}
        id=res['image_id']
        which=images_dict[id]
        try:
            d['x'] = which[0] + res['bbox'][0]
            d['y'] = which[1] + res['bbox'][1]
            d['w'] = res['bbox'][2]
            d['h'] = res['bbox'][3]
            d['p'] = res['score']
            d['class'] = categories_dict[res['category_id']]
            name = which[-1]
            count+=1
            if name in out2:
                out2[name].append(d)
    #             print(out)
            else:
                out2[name]=[d]
    #             print('No.%d :Original %s'%(i,name))
    #             print('e.g.',d)
                i+=1
        except:
            name=name.split("_")[1]
            if name not in lost:
                lost.append(name)
                print(lost)
            continue

    print('get outs,total: %d, cost time:%f'%(count,time.time()-start))
    
    return  out2
##1:'ASC-H', 2: 'ASC-US', 3: 'HSIL', 4: 'LSIL', 5: 'Candida', 6: 'Trichomonas'
def suan(out_file,Threshold = -0.05):
    n1 = 0
    n2 = 0
    n3 = 0
    n4 = 0
    n5 = 0
    n6 = 0
    for key in out_file:
        for value in out_file[key]:
            if value['p'] <= Threshold:    ###修改
                continue
            if value['class'] == 'ASC-H':
                n1 += 1
            elif value['class'] == 'ASC-US':
                n2 += 1
            elif value['class'] == 'HSIL':
                n3 += 1
            elif value['class'] == 'LSIL':
                n4 += 1
            elif value['class'] == 'Candida':
                n5 += 1
            elif value['class'] == 'Trichomonas':
                n6 += 1
            else:
                print("cuo")
    print(n1,n2,n3,n4,n5,n6)
def model_fusion_he (*args):
    out_file = {}
    out_name = []
    for outA in args:
        for name,bboxes in outA.items():
            bbox1 = chuli5 (bboxes)
            ###过滤0
            bbox_s = []
            for js1 in bbox1:
                if js1['p']>0.05:
                     bbox_s.append(js1) 
            if name not in out_name:
                out_file[name] = bbox_s
                out_name.append(name)
            else:
                for bbox in bbox_s:
                    out_file[name].append(bbox)
#     for name,bboxes in outB.items():
#         bbox1 = chuli5 (bboxes)
#         ###过滤0
#         bbox_s = []
#         for js1 in bbox1:
#             if js1['p']>0.05:
#                  bbox_s.append(js1) 
#         if name not in out_name:
#             out_file[name] = bbox_s
#             out_name.append(name)
#         else:
#             for bbox in bbox_s:
#                 out_file[name].append(bbox)
    
    return out_file
####融合
def model_fusion_chuli (out_file):    
    for name,bboxes in out_file.items():
        tbboxes = bboxes
        for i,box in enumerate( bboxes):  
            pp = bboxes[i]["p"]
#             if pp==0:      ###后面的p 为0的没有选举权，但是有投票权
#                 continue
            vote = []
            for j,tbox in enumerate( tbboxes):
                if i == j: # >=
                    continue
#                 if tbox['p'] == 0:
#                     continue
                if box['class'] != tbox['class']:
                    continue
                ##l两筐没关系
                if box["x"]+box['w']<=tbox["x"] or box["y"]+box['h']<=tbox["y"] or box["x"]>=tbox["x"]+tbox["w"] or box["y"]>=tbox["y"]+tbox["h"]: 
                    continue
                ####得到 iou   
                in_h = min(box['y']+box['h'],tbox['y']+tbox['h'])-max(box['y'],tbox['y'])
                in_w = min(box['x']+box['w'],tbox['x']+tbox['w'])-max(box['x'],tbox['x'])
                union = box['w']*box['h']+tbox['w']*tbox['h']-in_h*in_w
                iou = in_h*in_w/union
                
                if iou >0.4:  # 证明两者有关系
                    vote.append(j)   
#                 elif iou >0.3 and pp >=0.4:  # 证明两者有关系
#                     vote.append(j)
                else:
                    pass
            if len(vote)>=3:   #投票 的 决定  1时是 2票   
#                 print("???")
                pass
#                 for kk in vote:
#                     bboxes[kk]["p"] = 0
            else:
                bboxes[i]["p"] = 0
                    
        
        ##过滤 0 
        bbox_s = []
        for js1 in bboxes:
            if js1['p']>0.05:
                 bbox_s.append(js1)
        out_file[name] = bbox_s
    return out_file


In [ ]:
out1 = get_out(results1)
out2 = get_out(results2)
out3 = get_out(results3)
out4 = get_out(results4)
out5 = get_out(results5)
out6 = get_out(results6)

In [ ]:
# tout = copy.deepcopy(out2)
out_file = model_fusion_he (out2,out3,out4,out5,out6,out1)
suan(out_file)
# out_file = model_fusion_he (out_file,out1)
# suan(out_file)

out_file = model_fusion_chuli (out_file)
suan(out_file)
out_file = model_fusion_he (out_file)
suan(out_file)

In [ ]:
##1:'ASC-H', 2: 'ASC-US', 3: 'HSIL', 4: 'LSIL', 5: 'Candida', 6: 'Trichomonas'
n1 = 0
n2 = 0
n3 = 0
n4 = 0
n5 = 0
n6 = 0
for key in out_file:
    for value in out_file[key]:
        if value['p'] <= 0.05:    ###修改
            continue
        if value['class'] == 'ASC-H':
            n1 += 1
        elif value['class'] == 'ASC-US':
            n2 += 1
        elif value['class'] == 'HSIL':
            n3 += 1
        elif value['class'] == 'LSIL':
            n4 += 1
        elif value['class'] == 'Candida':
            n5 += 1
        elif value['class'] == 'Trichomonas':
            n6 += 1
        else:
            print("cuo")
print(n1,n2,n3,n4,n5,n6)

In [ ]:
# 生成提交用json 文件
outpath='./out_jsons/'
if not os.path.exists(outpath):
    os.makedirs(outpath)
start=time.time()
for name,bboxes in out_file.items():
#     chuli (bboxes)
    outjson=os.path.join(outpath,name+'.json')
    with open(outjson,'w') as f:
        json.dump(bboxes,f)
print('get nms outs,p_non_zero , cost time:%f'%(time.time()-start))

In [ ]:
#  检查 补全文件
outpath='./out_jsons/'
names = os.listdir(outpath)
tests=glob.glob(outpath+'*.json')
if os.path.exists(outpath+'.ipynb_checkpoints'):
#     os.removedirs(outpath+'.ipynb_checkpoints')
    names.remove('.ipynb_checkpoints')
path1 = r"/home/admin/jupyter/Data/test/"
tests=glob.glob(path1+'*.json')
print("结果文件个数",len(names),'should be ',len(tests))
c=1
for each in tests:              
    name=each.split('/')[-1]
    if name in names:
        pass
#         print('\texist no.%d :%s'%(c,name))
    else:
        with open(outpath + name, "w") as f:
            f.write('[]')
        print('add no.%d :%s'%(c,name))
    c+=1

In [ ]:
### 阴阳 分离
import os
import json
path = "./out_jsons_6/"
num = 0
for name in os.listdir(path)[1:]:
    with open(path+name,"r") as f:
        js = json.load(f) 
    no = 0
    yes1 = 0
    yes2 = 0
    for js1 in js:
        if js1["class"] =="Trichomonas":
            yes1 +=1
        elif js1["class"] =="Candida":
            yes2 +=1
        else:
            no +=1
    print(name,"异常",no,"滴虫",yes1,"念珠菌",yes2)
#     if no> yes1 and no > yes2:
#         for i,js1 in enumerate(js):
#             if js1["class"] =="Trichomonas":
#                 js[i]["p"] =0
#             elif js1["class"] =="Candida":
#                 js[i]["p"] = 0
#             else:
#                 pass
#         num += no
                
#     if yes1>no*4 and yes1 > yes2:
#         for i,js1 in enumerate(js):
#             if js1["class"] =="Trichomonas":
#                 pass
#             elif js1["class"] =="Candida":
#                 js[i]["p"] = 0
#             else:
#                 js[i]["p"] = 0
#         num += yes1
#     if yes2>no and yes2 > yes1:
#         for i,js1 in enumerate(js):
#             if js1["class"] =="Trichomonas":
#                 js[i]["p"] = 0  
#             elif js1["class"] =="Candida":
#                 pass
#             else:
#                 js[i]["p"] = 0
#         num += yes2
        
#     new_js = []
#     for js1 in js:
#         if js1['p']>0.05:   ####自动 过滤
#              new_js.append(js1) 
#     with open(path+name,"w") as f:
#         json.dump(new_js,f)

In [ ]:
##1:'ASC-H', 2: 'ASC-US', 3: 'HSIL', 4: 'LSIL', 5: 'Candida', 6: 'Trichomonas'
n1 = 0
n2 = 0
n3 = 0
n4 = 0
n5 = 0
n6 = 0
path = "./out_jsons/"
for name in os.listdir(path)[1:]:
    with open(path+name,"r") as f:
        js = json.load(f) 
    if len(js)==0:
        continue
    for value in js:
        if value['class'] == 'ASC-H':
            n1 += 1
        elif value['class'] == 'ASC-US':
            n2 += 1
        elif value['class'] == 'HSIL':
            n3 += 1
        elif value['class'] == 'LSIL':
            n4 += 1
        elif value['class'] == 'Candida':
            n5 += 1
        elif value['class'] == 'Trichomonas':
            n6 += 1
        else:
            print("cuo")
print(n1,n2,n3,n4,n5,n6)